# load data from Synapse ADI, perform a quaternion rotation.

In [1]:
# Baihan Lin
# Columbia University

In [2]:
%reset
%matplotlib inlinec

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


KeyError: 'inlinec'

In [8]:
import synapseclient
import numpy as np
import pandas as pd
import matplotlib.pylab as pl
import json
# import peakutils
# from peakutils.plot import plot as pplot
from scipy import signal, ndimage, io, stats
import scipy.integrate as integrate
from math import factorial
from scipy.stats import mode

In [9]:
# Login into the Synapse Client

# syn = synapseclient.Synapse()
syn = synapseclient.login()

# Enter login and password
#syn.login('login','password')

Welcome, Baihan Lin!



# Query demographics table

In [10]:
# demographicstable = syn.tableQuery('select * from syn5511429')

INPUT_DEMO_SYNID = "syn10146552"

demo_syntable = syn.tableQuery("SELECT * FROM syn10146552")
demo = demo_syntable.asDataFrame()
healthCodeList = ", ".join( repr(i) for i in demo["healthCode"]) 


# Query multiple datasets, starting from DeviceMotion

In [13]:
INPUT_WALKING_ACTIVITY_TABLE_SYNID = "syn10146553"
walkingtable = syn.tableQuery(('select * FROM {0} WHERE healthCode IN ({1}) AND "deviceMotion_walking_outbound.json.items" is not null LIMIT 500').format(INPUT_WALKING_ACTIVITY_TABLE_SYNID, healthCodeList))                              
walking_df = walkingtable.asDataFrame()
walking_df['idx'] = walking_df.index
walking_df = walking_df.drop(['createdOn', 'appVersion', 'phoneInfo','accel_walking_outbound.json.items',
                              'pedometer_walking_outbound.json.items','accel_walking_return.json.items',
                              'deviceMotion_walking_return.json.items','pedometer_walking_return.json.items',
                              'accel_walking_rest.json.items'],axis=1)

walking_df = walking_df.dropna()

filePaths_DMoutbound = syn.downloadTableColumns(walkingtable, ['deviceMotion_walking_outbound.json.items'])
filePaths_DMrest = syn.downloadTableColumns(walkingtable, ['deviceMotion_walking_rest.json.items'])

/Users/DoerLBH/anaconda/lib/python3.6/site-packages/synapseclient/client.py:3226: UserWarning: Weird file handle: None
  warnings.warn("Weird file handle: %s" % file_handle_id)


# store DeviceMotion datasets in dataframes and pickle them

In [12]:
demographics_df=demographicstable.asDataFrame()

wlist = np.arange(len(walking_df))

DMoutboundPathlist=[filePaths_DMoutbound[str(int(walking_df.ix[entry,'deviceMotion_walking_outbound.json.items']))] for entry in wlist] 
DMrestPathlist = [filePaths_DMrest[str(int(walking_df.ix[entry,'deviceMotion_walking_rest.json.items']))] for entry in wlist]

walking_df['DMoutboundPaths'] = DMoutboundPathlist
walking_df['DMrestPaths'] = DMrestPathlist

demographics_df.to_pickle('demographics_df.pkl')
walking_df.to_pickle('walking_df.pkl')

NameError: name 'demographicstable' is not defined

# perform a quaternion rotation on the accelerometer files

In [ ]:
# This function makes sure that the each quaternion in the angular phone information vector is properly normalized. The
# magnitude should be close to 1 within some tolerance. If it is not, rescale the quaternion.

def normalize(v, tolerance=0.00001):
    mag2 = sum(n * n for n in v)
    if abs(mag2 - 1.0) > tolerance:
        mag = sqrt(mag2)
        v = tuple(n / mag for n in v)
    return v


In [ ]:
# This funciton returns the conjugate of the quaternion

def get_quaternion_conjugate(quaternion):
    w, x, y, z = quaternion
    return (w, -x, -y, -z)

In [ ]:
# This function multiples two quaternions

def multiply_quaternions(quaternion1, quaternion2):
    w1, x1, y1, z1 = quaternion1
    w2, x2, y2, z2 = quaternion2
    # rearranged a little for my own clarity
    w = (w1 * w2) - (x1 * x2) - (y1 * y2) - (z1 * z2)
    x = (w1 * x2) + (x1 * w2) + (y1 * z2) - (z1 * y2)
    y = (w1 * y2) - (x1 * z2) + (y1 * w2) + (z1 * x2)
    z = (w1 * z2) + (x1 * y2) - (y1 * x2) + (z1 * w2)  
    return w, x, y, z

def multiply_quaternionANDvector(q1, v1):
    q1 = normalize(q1,tolerance = 0.00001) 
    q2 = (0.0,) + v1
    return multiply_quaternions(multiply_quaternions(q1, q2), get_quaternion_conjugate(q1))[1:]

# combine into a datastructure from all datasets

In [ ]:
dfdata= {'time':[],'rotX':[],'rotY':[],'rotZ':[],'total_rawacceleration':[],'rest_time':[],'rest_rotX':[],'rest_rotY':[],'rest_rotZ':[],'rest_total_rawacceleration':[],'healthCode':[]}
Accelerometer_df = pd.DataFrame(dfdata, columns =['time','rotX','rotY','rotZ','total_rawacceleration','rest_time','rest_rotX','rest_rotY','rest_rotZ','rest_total_rawacceleration','healthCode'])

# load accelerometer data

In [ ]:
mypkls =['0_2500','2500_5000','5000_7500','7500_10000','10000_12500','12500_15000','15000_17500',
         '17500_20000','20000_22500','22500_23093']

for entry in range(0,2500):
    
    if entry % 250 == 0:
        print entry
        
    healthCode = walking_df['healthCode'][entry]
    recordId = walking_df['recordId'][entry]
    medTimepoint = walking_df['medTimepoint'][entry]
    walkPath = walking_df['DMoutboundPaths'][entry]
    restPath = walking_df['DMrestPaths'][entry]

    walkingrecord = [json.loads(line) for line in open(walkPath)]
    restrecord = [json.loads(line) for line in open(restPath)]
    
    walkingrecordlist=walkingrecord[0]
    restrecordlist=restrecord[0]
    
    barlist = np.arange(len(walkingrecordlist))
    restbarlist = np.arange(len(restrecordlist))
    
    time = []
    acceleration_vector = []
    attitude_quaternion = []
    rest_acceleration_vector = []
    attitude_quaternion = []
    
    time = [(walkingrecordlist[bar]['timestamp']) for bar in barlist]
    acceleration_vector = [(walkingrecordlist[bar]['userAcceleration']['x'],
                            walkingrecordlist[bar]['userAcceleration']['y'],
                            walkingrecordlist[bar]['userAcceleration']['z']) for bar in barlist]
    attitude_quaternion = [(walkingrecordlist[bar]['attitude']['w'],
                            walkingrecordlist[bar]['attitude']['x'],
                            walkingrecordlist[bar]['attitude']['y'],
                            walkingrecordlist[bar]['attitude']['z']) for bar in barlist]
    
    rest_time = [(restrecordlist[restbar]['timestamp']) for restbar in restbarlist]
    rest_acceleration_vector = [(restrecordlist[restbar]['userAcceleration']['x'],
                            restrecordlist[restbar]['userAcceleration']['y'],
                            restrecordlist[restbar]['userAcceleration']['z']) for bar in restbarlist]
    rest_attitude_quaternion = [(restrecordlist[restbar]['attitude']['w'],
                            restrecordlist[restbar]['attitude']['x'],
                            restrecordlist[restbar]['attitude']['y'],
                            restrecordlist[restbar]['attitude']['z']) for restbar in restbarlist]
    
    # Calculate quadratic mean of original acceleration signal - this will be used as another directionless "axis" in my feature calculations
    [X, Y, Z] = zip(*acceleration_vector) 
    total_rawacceleration = np.sqrt(np.square(X)+np.square(Y)+np.square(Z))
    [restX, restY, restZ] = zip(*rest_acceleration_vector) 
    rest_total_rawacceleration = np.sqrt(np.square(restX)+np.square(restY)+np.square(restZ))
    
    # Apply quaternion rotations here
    plist = np.arange(len(acceleration_vector))
    quatrot = [multiply_quaternionANDvector(attitude_quaternion[p],acceleration_vector[p]) for p in plist]
    [rotX, rotY, rotZ] = zip(*quatrot) 
    
    rlist = np.arange(len(rest_acceleration_vector))
    rest_quatrot = [multiply_quaternionANDvector(rest_attitude_quaternion[r],rest_acceleration_vector[r]) for r in rlist]
    [rest_rotX, rest_rotY, rest_rotZ] = zip(*rest_quatrot) 
    
    dfdata= {'time':[],'rotX':[],'rotY':[],'rotZ':[],'total_rawacceleration':[],'rest_time':[],'rest_rotX':[],'rest_rotY':[],'rest_rotZ':[],'rest_total_rawacceleration':[],'healthCode':[]}

    Accelerometer_df = Accelerometer_df.append({'time':time,'rotX':rotX,'rotY':rotY,'rotZ':rotZ,
                                                'total_rawacceleration':total_rawacceleration,'rest_time':rest_time,
                                                'rest_rotX':rest_rotX,'rest_rotY':rest_rotY,'rest_rotZ':rest_rotZ,
                                                'rest_total_rawacceleration':rest_total_rawacceleration,
                                                'healthCode':healthCode, 'recordId':recordId,'medTimepoint':medTimepoint},ignore_index=True)
    

# cross-reference accelerometer data with demographics data and pickle

In [ ]:
# add relevant demographics data to walking activity feature data frame
Demographics_df = demographics_df.drop(['recordId','createdOn','appVersion','phoneInfo','are-caretaker', 'deep-brain-stimulation',
                                        'education','employment','health-history','healthcare-provider',
                                        'home-usage','last-smoked','maritalStatus','medical-usage',
                                        'medical-usage-yesterday','packs-per-day',
                                        'past-participation','phone-usage','race','smartphone',
                                        'smoked','surgery','video-usage','years-smoking',
                                        'diagnosis-year','medication-start-year','onset-year'],axis=1)
Combined_df = pd.merge(Accelerometer_df, Demographics_df, on='healthCode')

Final_df.to_pickle('Accelerometer_Demographics_df')

In [ ]:
##############################################################################
# Plot a sample time series



pl.figure()
pl.plot(fpr, tpr, 'g',label='ROC curve (area = %0.2f)' % roc_auc)
pl.plot([0, 1], [0, 1], 'k--')
pl.xlim([0.0, 1.0])
pl.ylim([0.0, 1.05])
pl.xticks(color = 'k', size = 28)
pl.yticks(color = 'k', size = 28)
pl.xlabel('False Positive Rate',{'color':'k','fontsize': 28})
pl.ylabel('True Positive Rate',{'color':'k','fontsize': 28})
pl.title('Receiver operating characteristic',{'color':'k','fontsize': 28})
pl.legend(loc="lower right",fontsize= 18)
pl.show()

In [ ]:
# sampling interval and frequency
samplingint = 0.01
Fs = 1/samplingint

In [ ]:
# ENTER FILENAME!
filename = 'Accelerometer_Demographics_df'

thresholdingparam = 1.5
peakfindinterval = 80

minnumpeaksfound = 5
maxnumpeaksfound = 20
numpeakintervalstoavg = 5

maxpoweratrest = 0.00005

FFTsamples = 1024
minsignallength = 1200

RESTFFTsamples = 512
RESTminsignallength = 515

# buffer for epochs between steps
buff = 5
minchunksamples = 90

# Accelerometer_df = pd.read_pickle(filename)
recordnums = Accelerometer_df.index # important - some indices won't be there since we dropped records before

# Function: Savitzky Golay Filter

In [ ]:
# Savitzky Golay filter from http://scipy.github.io/old-wiki/pages/Cookbook/SavitzkyGolay
def savitzky_golay(y, window_size, order, deriv=0, rate=1):
    try:
        window_size = np.abs(np.int(window_size))
        order = np.abs(np.int(order))
    except ValueError, msg:
        raise ValueError("window_size and order have to be of type int")
    if window_size % 2 != 1 or window_size < 1:
        raise TypeError("window_size size must be a positive odd number")
    if window_size < order + 2:
        raise TypeError("window_size is too small for the polynomials order")
    order_range = range(order+1)
    half_window = (window_size -1) // 2
    # precompute coefficients
    b = np.mat([[k**i for i in order_range] for k in range(-half_window, half_window+1)])
    m = np.linalg.pinv(b).A[deriv] * rate**deriv * factorial(deriv)
    # pad the signal at the extremes with values taken from the signal itself
    firstvals = y[0] - np.abs( y[1:half_window+1][::-1] - y[0] )
    lastvals = y[-1] + np.abs(y[-half_window-1:-1][::-1] - y[-1])
    y = np.concatenate((firstvals, y, lastvals))
    return np.convolve( m[::-1], y, mode='valid')

# Function: FFT

In [ ]:
def get_FFT(timeseries, samplingint):
    fouriercoefficients = np.fft.rfft(timeseries)
    normalization = 2.0/len(timeseries)
    fouriercoefficients = fouriercoefficients*normalization
    frequencies = np.fft.rfftfreq(len(timeseries),samplingint)
    return frequencies, fouriercoefficients

# Function: Find Peaks with PeakUtils threshold peak finder

In [ ]:
def get_peak_info(timesignal,processedsignal,sampling_int,thresholdingparam,peakfindinterval):
    AVGsignal = np.mean(processedsignal)
    minthreshold = thresholdingparam*AVGsignal
    peakindices = peakutils.indexes(processedsignal ,thres=minthreshold,min_dist=peakfindinterval)
    peaktimes = peakindices*sampling_int
    peakamplitudes = timesignal[peakindices]
    
    return peakindices, peaktimes, peakamplitudes

# Initialize dataframe for storing results

In [ ]:
featureinfo = {'healthCode':[],'record':[],'age':[],'gender':[],'professional-diagnosis':[],'recordId':[],
               'totpowerX':[],'totpowerY':[],'totpowerZ':[],
               'powentropyX':[], 'powentropyY':[],'powentropyZ':[],
               'numpeaksX':[],'numpeaksY':[],'numpeaksZ':[],
               'MEANpeakintX':[],'MEANpeakintY':[],'MEANpeakintZ':[],
               'CVpeakintX':[],'CVpeakintY':[],'CVpeakintZ':[],'duration':[],
               'FFT1Z':[],'FFT2Z':[],'FFT3Z':[],
               'modeFFT1chunksX':[], 'SDFFT1chunksX':[], 'modeFFT2chunksX':[],'SDFFT2chunksX':[], 'modeFFT3chunksX':[], 'SDFFT3chunksX':[],
               'meanpowchunksX':[],'CVpowchunksX':[],'meanentropychunksX':[],'CVentropychunksX':[],
               'modeFFT1chunksY':[], 'SDFFT1chunksY':[],'modeFFT2chunksY':[], 'SDFFT2chunksY':[], 'modeFFT3chunksY':[], 'SDFFT3chunksY':[],
               'meanpowchunksY':[],'CVpowchunksY':[],'meanentropychunksY':[],'CVentropychunksY':[],
               'modeFFT1chunksZ':[], 'SDFFT1chunksZ':[], 'modeFFT2chunksZ':[], 'SDFFT2chunksZ':[], 'modeFFT3chunksZ':[], 'SDFFT3chunksZ':[],
               'meanpowchunksZ':[],'CVpowchunksZ':[],'meanentropychunksZ':[],'CVentropychunksZ':[],
               'onset_lag':[],'sumabs_acceleration':[]}

feature_df = pd.DataFrame(featureinfo,columns =['healthCode','record','age', 'gender','professional-diagnosis','recordId',
                                                'totpowerX','totpowerY','totpowerZ',
                                                'powentropyX','powentropyY','powentropyZ',
                                                'numpeaksX','numpeaksY','numpeaksZ',
                                                'MEANpeakintX','MEANpeakintY','MEANpeakintZ',
                                                'CVpeakintX','CVpeakintY','CVpeakintZ',
                                                'duration','FFT1Z','FFT2Z','FFT3Z',
                                                'modeFFT1chunksX','modeFFT2chunksX','SDFFT1chunksX', 'SDFFT2chunksX', 'modeFFT3chunksX', 'SDFFT3chunksX', 
                                                'meanpowchunksX','CVpowchunksX','meanentropychunksX','CVentropychunksX',
                                                'modeFFT1chunksY', 'SDFFT1chunksY', 'modeFFT2chunksY', 'SDFFT2chunksY', 'modeFFT3chunksY', 'SDFFT3chunksY',
                                                'meanpowchunksY','CVpowchunksY','meanentropychunksY','CVentropychunksY',
                                                'modeFFT1chunksZ', 'SDFFT1chunksZ','modeFFT2chunksZ', 'SDFFT2chunksZ', 'modeFFT3chunksZ', 'SDFFT3chunksZ', 
                                                'meanpowchunksZ','CVpowchunksZ','meanentropychunksZ','CVentropychunksZ',
                                                'onset_lag','sumabs_acceleration'])

# Function: calculate features for the recording

In [ ]:
def featuregrabber(record,signame,sig,datamat,datamat_YSG,datamat_processed,samplingint,thresholdingparam,peakfindinterval,minsignallength,minnumpeaksfound,buff):
    
    # initialize lists for storing calculations for each epoch or "chunk" between steps
    FFT1chunks = []
    FFT2chunks = []
    FFT3chunks = []
    powerchunks = []
    entropychunks = []
    
    # calculate the duration of the recording
    duration = (len(datamat[sig]))*samplingint
    
    # take recordings that are longer than the minimal length
    if len(datamat[sig]) >= minsignallength:
        [peakindices, peaktimes, peakamplitudes] = get_peak_info(datamat[sig],datamat_processed[sig],samplingint,thresholdingparam,peakfindinterval)
        stepintervals = np.diff(peaktimes)
        numpeaks = len(peakindices)

        # take recordings where the number of peaks discovered is within the desired range
        if numpeaks > minnumpeaksfound and numpeaks <= maxnumpeaksfound:
            onset_lag = peakindices[0]*samplingint
            for k in np.arange(numpeaks-2):
                peakstart = peakindices[k]+buff
                peakend = peakindices[k+1]-buff
                chunklen = peakend-peakstart
                midchunk = peakstart+int(chunklen/2)
                if chunklen > minchunksamples:
                    chunk = datamat_YSG[sig][midchunk-(minchunksamples/2):midchunk+(minchunksamples/2)]
                    fYSG_chunk,Pxx_denYSG_chunk = signal.welch(chunk, Fs)#nperseg=128)
                    
                    freqchunk, ffcoeffchunk = get_FFT(chunk,samplingint)
                    ffcoeffchunk = np.absolute(ffcoeffchunk)**2
            
                    ff_chunkdf = pd.DataFrame({'fouriercoefficients':ffcoeffchunk,'frequencies':freqchunk})
                    ff_chunkdf=ff_chunkdf.sort_values('fouriercoefficients',ascending = False)
            
                    if ff_chunkdf.iloc[0]['frequencies'] != 0:
                        FFT1chunks.append(ff_chunkdf.iloc[0]['frequencies'])
                        FFT2chunks.append(ff_chunkdf.iloc[1]['frequencies'])
                        FFT3chunks.append(ff_chunkdf.iloc[2]['frequencies'])
                        totchunkpower = np.trapz(Pxx_denYSG_chunk)

                        entropychunk = stats.entropy(Pxx_denYSG_chunk/totchunkpower)
                        powerchunks.append(totchunkpower)
                        entropychunks.append(entropychunk)
                        
            
            # take a central part of the recording (same length for all recordings), and calculate frequency spectrum features
            midsignal = int(len(datamat_YSG[sig])/2)
            FFTsignal = datamat_YSG[sig][(midsignal-FFTsamples/2):(midsignal+FFTsamples/2)]
            
            f,Pow = signal.welch(FFTsignal, Fs,scaling='density')
            totpower = np.trapz(Pow)
            powentropy = stats.entropy(Pow/totpower)
            
   
            frequencies, fouriercoefficients = get_FFT(FFTsignal,samplingint)
            fouriercoefficients = np.absolute(fouriercoefficients)**2
            ff_df = pd.DataFrame({'fouriercoefficients':fouriercoefficients,'frequencies':frequencies})
            ff_df = ff_df.sort_values('fouriercoefficients',ascending = False)
        
            if ff_df.iloc[0]['frequencies'] != 0:
                FFT1 = ff_df.iloc[0]['frequencies']
                FFT2 = ff_df.iloc[1]['frequencies']
                FFT3 = ff_df.iloc[2]['frequencies']
            
            else:
                FFT1 = float('nan')
                FFT2 = float('nan')
                FFT3 = float('nan')
        else:
            onset_lag = float('nan')
            totpower = float('nan')
            powentropy = float('nan')
            FFT1 = float('nan')
            FFT2 = float('nan')
            FFT3 = float('nan')
     
    else:
        numpeaks = float('nan')
        stepintervals = float('nan')
        totpower = float('nan')
        powentropy = float('nan')
        FFT1 = float('nan')
        FFT2 = float('nan')
        FFT3 = float('nan')
        onset_lag = float('nan')
        
    return duration, onset_lag, numpeaks, stepintervals, totpower, powentropy, FFT1, FFT2, FFT3, FFT1chunks, FFT2chunks, FFT3chunks, powerchunks, entropychunks

# Function: calculate features within epochs or "chunks" between steps

In [ ]:
def chunkstats(FFT1chunks, FFT2chunks, FFT3chunks, powerchunks, entropychunks):
    if len(FFT1chunks)>=1:
        modeFFT1chunks = mode(FFT1chunks)[0][0]
        meanpowchunks = np.mean(powerchunks)
        SDFFT1chunks = np.std(FFT1chunks)
        CVpowchunks = np.std(powerchunks) / meanpowchunks
        meanentropychunks = np.mean(entropychunks)
        CVentropychunks = np.std(entropychunks) / meanentropychunks
        
    else:
        modeFFT1chunks = float('nan')  
        meanpowchunks = float('nan')
        SDFFT1chunks = float('nan')
        CVpowchunks = float('nan')
        meanentropychunks = float('nan')
        CVentropychunks = float('nan')
        
    if len(FFT2chunks)>=1:
        modeFFT2chunks = mode(FFT2chunks)[0][0]
        SDFFT2chunks = np.std(FFT2chunks)
        
    else:
        modeFFT2chunks = float('nan')  
        SDFFT2chunks = float('nan')
        
            
    if len(FFT3chunks)>=1:
        modeFFT3chunks = mode(FFT3chunks)[0][0]
        SDFFT3chunks = np.std(FFT3chunks)
        
    else:
        modeFFT3chunks = float('nan')  
        SDFFT3chunks = float('nan')
        
    return modeFFT1chunks, SDFFT1chunks, modeFFT2chunks, SDFFT2chunks, modeFFT3chunks, SDFFT3chunks, meanpowchunks,CVpowchunks, meanentropychunks,CVentropychunks  

# Go through each record in the data frame, calculate features and store in a dataframe

In [ ]:
for record in recordnums:
    
    ##################################### WALKING DATA ###################################
    
    time = Accelerometer_df['time'][record]
    datamat = np.array([np.asarray(Accelerometer_df['rotX'][record]), np.asarray(Accelerometer_df['rotY'][record]),
                        np.asarray(Accelerometer_df['rotZ'][record]), np.asarray(Accelerometer_df['total_rawacceleration'][record])])

    axisitems  = np.arange(0,len(datamat))
    
    # Apply Savitsky Golay filter
    datamat_YSG = [savitzky_golay(datamat[myaxis], window_size =51, order=4) for myaxis in axisitems]
    
    # Apply Savitsky Golay filter to squared signal for peak finding
    datamat_YSG_peaks = [savitzky_golay(np.square(datamat[myaxis]), window_size=51, order=4) for myaxis in axisitems]
    datamat_processed = datamat_YSG_peaks
     
    
    # X
    durationX, onset_lagX, numpeaksX, stepintervalsX, totpowerX, powentropyX, FFT1X, FFT2X, FFT3X, FFT1chunksX, FFT2chunksX, FFT3chunksX, powerchunksX, entropychunksX = featuregrabber(str(record),'X',0,datamat,datamat_YSG,datamat_processed,samplingint,thresholdingparam,peakfindinterval,minsignallength,minnumpeaksfound,buff)
    modeFFT1chunksX, SDFFT1chunksX, modeFFT2chunksX, SDFFT2chunksX, modeFFT3chunksX, SDFFT3chunksX, meanpowchunksX,CVpowchunksX,meanentropychunksX,CVentropychunksX  = chunkstats(FFT1chunksX, FFT2chunksX, FFT3chunksX, powerchunksX, entropychunksX)
    
    
    # Y
    durationY, onset_lagY, numpeaksY, stepintervalsY, totpowerY, powentropyY, FFT1Y, FFT2Y, FFT3Y, FFT1chunksY, FFT2chunksY, FFT3chunksY, powerchunksY, entropychunksY = featuregrabber(str(record),'Y',1,datamat,datamat_YSG,datamat_processed,samplingint,thresholdingparam,peakfindinterval,minsignallength,minnumpeaksfound,buff)
    modeFFT1chunksY, SDFFT1chunksY, modeFFT2chunksY, SDFFT2chunksY, modeFFT3chunksY, SDFFT3chunksY, meanpowchunksY,CVpowchunksY,meanentropychunksY,CVentropychunksY  = chunkstats(FFT1chunksY, FFT2chunksY, FFT3chunksY, powerchunksY, entropychunksY)
    

    # Z
    durationZ, onset_lagZ, numpeaksZ, stepintervalsZ, totpowerZ, powentropyZ, FFT1Z, FFT2Z, FFT3Z, FFT1chunksZ, FFT2chunksZ, FFT3chunksZ, powerchunksZ, entropychunksZ = featuregrabber(str(record),'Z',2,datamat,datamat_YSG,datamat_processed,samplingint,thresholdingparam,peakfindinterval,minsignallength,minnumpeaksfound,buff)
    modeFFT1chunksZ, SDFFT1chunksZ, modeFFT2chunksZ, SDFFT2chunksZ, modeFFT3chunksZ, SDFFT3chunksZ, meanpowchunksZ,CVpowchunksZ,meanentropychunksZ,CVentropychunksZ  = chunkstats(FFT1chunksZ, FFT2chunksZ, FFT3chunksZ, powerchunksZ, entropychunksZ)
    
    
    # QUADRATIC SUM OF X,Y,Z
    durationSUM, onset_lagSUM,numpeaksSUM, stepintervalsSUM, totpowerSUM, powentropySUM, FFT1SUM, FFT2SUM, FFT3SUM, FFT1chunksSUM, FFT2chunksSUM, FFT3chunksSUM, powerchunksSUM, entropychunksSUM = featuregrabber(str(record),'SUM',3,datamat,datamat_YSG,datamat_processed,samplingint,thresholdingparam,peakfindinterval,minsignallength,minnumpeaksfound,buff)
    modeFFT1chunksSUM, SDFFT1chunksSUM, modeFFT2chunksSUM, SDFFT2chunksSUM, modeFFT3chunksSUM, SDFFT3chunksSUM, meanpowchunksSUM,CVpowchunksSUM,meanentropychunksSUM,CVentropychunksSUM = chunkstats(FFT1chunksSUM, FFT2chunksSUM, FFT3chunksSUM, powerchunksSUM, entropychunksSUM)
    
    
    
    # Get metadata for the record
    healthCode = Accelerometer_df['healthCode'][record]
    age = Accelerometer_df['age'][record]
    gender = Accelerometer_df['gender'][record]
    professional_diagnosis = Accelerometer_df['professional-diagnosis'][record]
    recordId = Accelerometer_df['recordId'][record]
    
    # Make final feature calculations
    MEANpeakintX = np.mean(stepintervalsX)
    MEANpeakintY = np.mean(stepintervalsY)
    MEANpeakintZ = np.mean(stepintervalsZ)
    CVpeakintX = np.std(stepintervalsX) / MEANpeakintX
    CVpeakintY = np.std(stepintervalsY) / MEANpeakintY
    CVpeakintZ = np.std(stepintervalsZ) / MEANpeakintZ
    duration = durationZ
    onset_lag = onset_lagZ
    sumabs_acceleration = sum(abs(datamat[3]))

   
    feature_df = feature_df.append({'healthCode':healthCode,'record':record,'age':age, 'gender':gender, 'professional-diagnosis':professional_diagnosis,'recordId':recordId, 
                                    'totpowerX':totpowerX,'totpowerY':totpowerY,'totpowerZ':totpowerZ,
                                    'powentropyX':powentropyX, 'powentropyY':powentropyY,'powentropyZ':powentropyZ,
                                    'numpeaksX':numpeaksX,'numpeaksY':numpeaksY,'numpeaksZ':numpeaksZ,
                                    'MEANpeakintX':MEANpeakintX,'MEANpeakintY':MEANpeakintY,'MEANpeakintZ':MEANpeakintZ,
                                    'CVpeakintX':CVpeakintX,'CVpeakintY':CVpeakintY,'CVpeakintZ':CVpeakintZ,
                                    'duration':duration,'FFT1Z':FFT1Z,'FFT2Z':FFT2Z,'FFT3Z':FFT3Z,  
                                    'modeFFT1chunksX':modeFFT1chunksX,'SDFFT1chunksX':SDFFT1chunksX,
                                    'modeFFT2chunksX':modeFFT2chunksX,'SDFFT2chunksX':SDFFT2chunksX,
                                    'modeFFT3chunksX':modeFFT3chunksX,'SDFFT3chunksX':SDFFT3chunksX,
                                    'meanpowchunksX':meanpowchunksX,'CVpowchunksX':CVpowchunksX,
                                    'meanentropychunksX':meanentropychunksX,'CVentropychunksX':CVentropychunksX,
                                    'modeFFT1chunksY':modeFFT1chunksY,'SDFFT1chunksY':SDFFT1chunksY,
                                    'modeFFT2chunksY':modeFFT2chunksY,'SDFFT2chunksY':SDFFT2chunksY,
                                    'modeFFT3chunksY':modeFFT3chunksY,'SDFFT3chunksY':SDFFT3chunksY,
                                    'meanpowchunksY':meanpowchunksY,'CVpowchunksY':CVpowchunksY,
                                    'meanentropychunksY':meanentropychunksY,'CVentropychunksY':CVentropychunksY,
                                    'modeFFT1chunksZ':modeFFT1chunksZ,'SDFFT1chunksZ':SDFFT1chunksZ,
                                    'modeFFT2chunksZ':modeFFT2chunksZ,'SDFFT2chunksZ':SDFFT2chunksZ,
                                    'modeFFT3chunksZ':modeFFT3chunksZ,'SDFFT3chunksZ':SDFFT3chunksZ,
                                    'meanpowchunksZ':meanpowchunksZ,'CVpowchunksZ':CVpowchunksZ,
                                    'meanentropychunksZ':meanentropychunksZ,'CVentropychunksZ':CVentropychunksZ,
                                    'onset_lag':onset_lag,'sumabs_acceleration':sumabs_acceleration},ignore_index=True)  
    

# Remove NANs and pickle the resulting dataframe

In [ ]:
cleanfeature_df = feature_dfmod.dropna()
cleanfeature_df.to_pickle('feature_df')